In [1]:
# import beautifulsoup4 to use
from bs4 import BeautifulSoup
import requests

In [2]:
def crawleAndFetchAssets():
    url = "https://neal.fun/ambient-chaos/"
    images = set()
    # fetch the content from url
    page = requests.get(url)

    # parse html using beautifulsoup
    soup = BeautifulSoup(page.content, 'html.parser')

    # find all the images in the page
    imgs = soup.findAll('img')

    # loop over all the images
    for img in imgs:
        # extract the link
        img_url = img['src']

        # replace '/ambient-chaos/icons' to ''
        img_url = img_url.replace('/ambient-chaos/icons/', '')

        # replace '.svg' to ''
        img_url = img_url.replace('.svg', '')

        # if image includes 'logo' then skip
        if 'logo' in img_url:
            continue

        # add the image url to the set
        images.add(img_url)

    return images

In [3]:
assets = crawleAndFetchAssets()

In [4]:
def downloadSound(asset=""):
    # download image from https://neal.fun/ambient-chaos/sounds/$asset.mp3  to folder ./sounds
    url = f"https://neal.fun/ambient-chaos/sounds/{asset}.mp3"

    # get the content of the url
    r = requests.get(url)

    # write the content to file
    with open(f"./sounds/{asset}.mp3", 'wb') as f:
        f.write(r.content)

In [5]:
# for asset in assets:
#     downloadSound(asset)

In [6]:
import openai
import dotenv
import os
import threading

In [7]:
# use dotenv
dotenv.load_dotenv()

True

In [8]:
# set openai api key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
# use openai dall-e model to generate image with size 256x256 to folder ./images
def generateImage(asset=""):
    # set the prompt
    prompt = f"imagine {asset}"

    # set the completion
    response = openai.images.generate(
        prompt=prompt,
        model="dall-e-2",
        size="256x256",
        style="vivid",
        response_format="url",
        quality="standard",
        timeout=100000
    )

    # get the image url
    img_url = response.data[0].url

    # write the image to file
    with open(f"./images/{asset}.png", 'wb') as f:
        f.write(requests.get(img_url).content)

    print(f"Generated image for {asset}")

In [10]:
# start councurrent threads to generate images
threads = []

for asset in assets:
    # if assets file not exists then generate image
    if not os.path.exists(f"./images/{asset}.png"):
        t = threading.Thread(target=generateImage, args=(asset,))
        t.start()
        threads.append(t)

# wait for all threads to finish
for t in threads:
    t.join()

In [11]:
import json

In [12]:
# read app.json file
app = {}

with open('./app.json') as f:
    app = json.load(f)

In [13]:
# add the assets if image exists from set to list
app['data']['sounds'] = []

for asset in assets:
    if os.path.exists(f"./images/{asset}.png"):
        app['data']['sounds'].append(asset)

In [14]:
# write app.json file
with open('./app.json', 'w') as f:
    f.write(json.dumps(app, indent=4))